# Лабораторная работа по Теории кодирования
## Лабораторная работа №5
Выполнили студенты группы 6401-010302D

Баловнева Юлия

Богатырев Дмитрий


In [210]:
from ast import Bytes
from venv import create
from xmlrpc.client import Boolean

import numpy as np
import itertools

from numpy import dtype
from numpy.f2py.auxfuncs import throw_error


def mult(m1, m2):
    '''Умножает две матрицы xor-ом и возвращает результат
    :param:
        m1: первая матрица
        m2: вторая матрица
        
        :return: C: возвращает матрицу - xor-произведение двух матриц 
    '''
    A = np.array(m1, dtype=int)
    B = np.array(m2, dtype=int)
    C = (A@B)%2
    return np.array(C, dtype=bool)

def code_length(matr):
    '''Вычисляет кодовое расстояние массива переданных слов
    :param:
        matr: матрица - двумерный bool-массив - масиив двоичных слов
        
        :return: d: кодовое расстояние. 
    '''
    d = len(matr[0])
    for i in range(len(matr)-1):
        for j in range(i+1,len(matr)):
            # Ищем минимальное число единиц в множестве произведений i и j строк
            d = min(d,sum(np.bitwise_xor(matr[i],matr[j])))
    return d

def create_verif_matr(X):
    k = X.shape[1]
    P = np.eye(k,dtype=bool)
    return np.vstack([P,X])

def create_rref_matr(X):
    k = X.shape[0]
    P = np.eye(k,dtype=bool)
    return np.hstack([P,X])

def kronekers_mult(A,B):
    i_stack = None
    for i in range(A.shape[0]):
        j_stack = None
        for j in range(A.shape[1]):
            aijB = A[i,j]*B
            if j_stack is None:
                j_stack = aijB
            else:
                j_stack = np.hstack((j_stack,aijB))
                
        if i_stack is None:
            i_stack = j_stack
        else:
            i_stack = np.vstack((i_stack,j_stack))
            
    return i_stack

def inner_shape(matr, dimension):
    ndims = matr.ndim
    if ndims == 1 and dimension == 1:
        return matr.shape[0]
    elif ndims == 1 and dimension == 0:
        return 1
    elif ndims == 2:
        return matr.shape[dimension]
    else:
        print("Unidefined dimension error")
        return -1
    
def to_binary_v(a, m):
    n = format(a, 'b')
    res = np.zeros(m, dtype=int)
    for i in range(len(n)):
        res[i] = int(n[len(n) - i - 1])
    return res


In [211]:
class LinearCode:
    ''' Класс линейных кодов.

    Атрибуты:\n
    matr: копия исходной матрицы\n
    k: количество строк матрицы\n
    n: количество столбцов матрицы
    
    '''
    # Конструктор
    def __init__(self, n_array):
        self.matr = np.array(n_array, dtype=bool)
        self.n = n_array.shape[1]
        self.k = n_array.shape[0]
    
    def find_first(self, line):
        '''Поиск первого ненулевого элемента в строке.
        
        :param:
        line: строка матрицы.
        
        :return: h: возвращаем позицию первого ненулевого элемента в строке. 
        '''
        # Задаем h = количеству элементов в сторке
        h = self.n
        # В строке line перебираем элементы пока не натолкнемся на 1
        for i in range(self.n):
            if self.matr[line,i]:
                h = i
                break
        # Возвращаем позицию найденной единицы в строке
        return h
    
    def xor_swap(self, line1, line2):
        '''Меняет две строки местами с помощью поэлементного XOR.

        :param:
            line1: первая строка.\n
            line2: вторая строка.

        '''
        # Поэлементный XOR 
        for w in range(self.n):
            self.matr[line1,w] ^= self.matr[line2,w]
            self.matr[line2,w] ^= self.matr[line1,w]
            self.matr[line1,w] ^= self.matr[line2,w]


    def sum(self, line1, line2):
        '''Складывает первую переданную строку со второй.

        :param:
            line1: первая строка.\n
            line2: вторая строка.
        '''
        # Поэлементный XOR 
        for w in range(self.n):
            self.matr[line2,w] ^= self.matr[line1,w]

    def __str__(self):
        '''Собирает строку (смена типа от bool к string).

        :return: s: строка с матрицей.            
        '''
        s = ''
        for i in range(self.k):
            s += '['
            for j in range(self.n):
                if self.matr[i,j]: s += "1 "
                else: s += "0 "
            s += "\b]\n"
        return s
    
    def fillrand(self):
        '''Заполняет матрицу случайным образом нулями и единицами.
        
        '''
        rand = np.random
        for i in range(self.k):
            for j in range(self.n):
                self.matr[i,j] = np.round(rand.random())

    def sort(self):
        '''Сортировка строк матрицы.
        '''
        # sort
        for i in range(self.k):
            for j in range(self.k-i-1):
                if self.find_first(j) > self.find_first(j+1):
                    self.xor_swap(j, j+1)

    def ref(self):
        '''Приводит матрицу к ступенчатому виду.
        
            ---
            Ступенчатая матрица - такая матрица, что
            * все ненулевые строки располагаются над всеми чисто нулевыми строками
            * ведущий элемент (первый, считая слева направо, ненулевой элемент строки)
            каждой ненулевой строки располагается строго правее ведущего элемента в
            строке, расположенной выше данной.
            ---
        '''
        # Проходит по всем строкм матрицы
        for i in range(self.k-1):
            # Сравнивает рабочую строку с текущей
            for j in range(i+1, self.k):
                # Запоминаем первые элементы
                first_i = self.find_first(i)
                first_j = self.find_first(j)
                # Если они равны, то делаем xor со строкой
                # Поскольку рабочая строка всегда ниже текущей, то при xor первый элемент всех нижних строк зануляется
                if first_i == first_j:
                    self.sum(i,j)
                # Если встретили строку, у которой первый элемент раньше чем у текущей, то меняем строки местами
                elif first_i > first_j:
                    self.xor_swap(i,j)
                # Иначе все хорошо, идем дальше
        # Удаляем занулившиеся строки, которые могли бы получится
        self.delete_redundant_lines()
        

    def rref(self):
        '''Приводит матрицу к приведенному ступенчатому виду. 
        
            ---
            Ступенчатая матрица называется приведенной, если матрица,  не имеет нулевых строк, 
            и все ведущие элементы ее строк равны единице. При этом все элементы основных столбцов, 
            помимо ведущих элементов, являются нулями.
            
        '''
        # Сначала приводим матрицу к ступенчатому виду
        self.ref()
        # Дальше для всех строк...
        for i in range(1,self.k):
            # ...мы ищем номер ведущего элемента строки...
            ist = self.find_first(i)
            # ...и делаем xor для всех строк выше текущей
            for j in range(i-1,-1,-1):
                if(self.matr[j,ist]):
                    self.sum(i,j)
            # Таким образом зануляются все элементы над ведущим
        # Удаляем занулившиеся строки, которые могли бы получится
        self.delete_redundant_lines()
    

    def delete_redundant_lines(self):
        '''Удаляет нулевые строки из матрицы.
        '''
        # Копия матрицы, с которой ведется работа
        temp = self.matr.copy()
        offset = 0 # сдвиг
        for i in range(self.k):
            # Если номер первой единицы в строке == кол-ву элементов самой строки (то есть единиц там нет)
            # Удаляем эту строку и сдвигаем указатель на 1
            if self.find_first(i) == self.n:
                temp = np.delete(temp, i - offset, 0)
                offset += 1
        # Перезаписываем матрицу на новую и очищенную
        self.matr = temp
        # Обновляем переменные-размеры матрицы
        self.n = temp.shape[1]
        self.k = temp.shape[0]

    
    def create_reduced_matr(self):
        rrefed_matr = LinearCode(self.matr.copy())
        rrefed_matr.rref()

        # Определяем индексы ведущих элементов
        leads = np.array([rrefed_matr.find_first(i) for i in range(rrefed_matr.k)])

        # Создаем сокращенную матрицу 
        X = np.zeros(shape=(rrefed_matr.k,rrefed_matr.n-rrefed_matr.k), dtype=bool)
        
        # Заполняем ее значениями из ступенчатой
        for i in range(rrefed_matr.k):
            offset = 0 # сдвиг
            for j in range(rrefed_matr.n):
                # Если в столбце содержится ведущий элемент, то его пропускаем (и увеличиваем сдвиг на 1)
                # Иначе добавляем этот элемент в сокращенную матрицу
                if j in leads: 
                    offset+=1
                else:
                    X[i,j-offset] = rrefed_matr.matr[i,j]
        return LinearCode(X)

                
    def create_verif_matr(self):
        '''Создает проверочную матрицу на основе порождающей.
        
        :return: LinearCode(H): проверочная матрица. 
        '''
        # Создаем новую матрицу в приведенном ступенчатом виде на основе текущей
        rrefed_matr = LinearCode(self.matr.copy())
        rrefed_matr.rref()

        # Определяем индексы ведущих элементов
        leads = np.array([rrefed_matr.find_first(i) for i in range(rrefed_matr.k)])

        # Создаем сокращенную матрицу 
        X = np.zeros(shape=(rrefed_matr.k,rrefed_matr.n-rrefed_matr.k), dtype=bool)
        
        # Заполняем ее значениями из ступенчатой
        for i in range(rrefed_matr.k):
            offset = 0 # сдвиг
            for j in range(rrefed_matr.n):
                # Если в столбце содержится ведущий элемент, то его пропускаем (и увеличиваем сдвиг на 1)
                # Иначе добавляем этот элемент в сокращенную матрицу
                if j in leads: 
                    offset+=1
                else:
                    X[i,j-offset] = rrefed_matr.matr[i,j]

        # Единичная матрица
        Ind = np.identity(n = rrefed_matr.n-rrefed_matr.k, dtype = bool)

        # Создаем проверочную матрицу
        H = np.zeros(shape=(rrefed_matr.n,rrefed_matr.n-rrefed_matr.k), dtype=bool)
        # сдвиги
        offset_i = 0
        offset_matr = 0
        # Заполняем ее значениями
        for i in range(H.shape[0]):
            # Если номер строки содержится в массиве с ведущими элементами, то заполняем строку из сокращенной матрицы
            # И прибавляем 1 к сдвигу для элементов единичной матрицы
            if i in leads:
                for j in range(H.shape[1]):
                    H[i,j] = X[i-offset_matr,j]
                offset_i += 1
            # Иначе заполняем строку из единичной матрицы
            # И прибавляем 1 к сдвику для матрицы сокращенной
            else:
                for j in range(H.shape[1]):
                    H[i,j] = Ind[i-offset_i,j]
                offset_matr += 1
        
        return LinearCode(H)
    
    def gen_codewords_summing(self):
        ''' Формирует набор слов путем сложения всех слов из порождающего множества

        :return: wordset: набор кодовых слов
        '''
        # Создаем пустой набор слов и доюавляем в него 0й элемент
        wordset = set()
        wordset.add(tuple((np.zeros(self.n, dtype=bool))))
        # Для i числа строк
        for i in range(1, self.k+1):
            # Массив длинны i всех возможных комбинаций строк 
            combinations = np.array(list(itertools.combinations(range(self.k), i)))
            for comb in combinations:
                # Для каждой комбинации
                # Создаем пустое слово
                word = np.zeros(self.n, dtype=bool)
                # Записываем в него сумму всех строк в текущей комбинации строк
                for j in comb:
                    word ^= self.matr[j]
                # Добавляем слово в набор слов
                wordset.add(tuple(word.tolist()))
        return wordset
    
    def gen_codewords_bin(self):
        ''' Формирует набор слов путем умножения всех двоичных слов длины k на G 

        :return: wordset: набор кодовых слов
        '''
        # Копия текущей матрицы
        G = LinearCode(self.matr)
        # Приводим в ступенчатому виду
        G.ref
        # Пустой набор слов
        wordset = set()
        # Набор всех возможных двоичных слов длины k
        binset = set(tuple(itertools.product((True,False),repeat=self.k)))
        # Для каждой комбинации
        for bin in binset:
            # Умножаем комбинацию на ступенчатую матрицу
            # И записываем слово в набор
            word = tuple((mult(bin,G.matr)).tolist())
            wordset.add(word)
        return wordset



### Часть 1
#### 5.1

In [212]:
def f(x_v,I):
        if len(I) == 0:
            return np.int64(1)
        if len(I) == 1:
            s = x_v[I[0]]^1
            return s
        s = 1
        for i in range(len(I)):
            s &= x_v[I[i]]^1
        return s
    

def vector_form(m,I):
    n = pow(2,m)
    v = list()
    for i in range(n):
        v.append(f(to_binary_v(i,m),I))
    return v
    
print(vector_form(4,[3]))

[np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]


In [220]:
def get_all_combs(n):
    all_sets = list()
    for i in(range(0, n)):
        combinations = list(itertools.combinations(range(n), i))
        combinations_rev = list(list(reversed(sorted([list(reversed(combinations[i])) for i in range(len(combinations))], reverse=True)[i])) for i in range(len(combinations))) # А что вы знаете о пайплайнах?
        for comb in combinations_rev:
            all_sets.append(comb)
    return all_sets


def get_all_combs_rm(r,m):
        all_sets = list()
        for i in(range(0, r+1)):
            combinations = list(itertools.combinations(range(m), i))
            combinations_rev = list(list(reversed(sorted([list(reversed(combinations[i])) for i in range(len(combinations))], reverse=True)[i])) for i in range(len(combinations))) # А что вы знаете о пайплайнах?
            for comb in combinations_rev:
                all_sets.append(comb)
        return all_sets


def create_G_for_RM(r,m):
    G = list()
    all_sets = get_all_combs_rm(r,m)    
    for i in range(len(all_sets)):
        G.append(vector_form(m,all_sets[i]))
    return np.array(G)

  
print("Z = ") 
print(get_all_combs_rm(3,4))

print("G = ")
create_G_for_RM(3,4)

Z = 
[[], [3], [2], [1], [0], [2, 3], [1, 3], [0, 3], [1, 2], [0, 2], [0, 1], [1, 2, 3], [0, 2, 3], [0, 1, 3], [0, 1, 2]]
G = 


array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

#### 5.2

In [214]:
def get_complimentary(I,Z):
    I = set(I)
    Z = set(Z)
    return list(Z-I)


def get_Hi(m,I):
    n = pow(2,m)
    Hi = list()
    for i in range(n):
        u = to_binary_v(i,m)
        if f(u,I) == 1:
            Hi.append(u)
    return Hi

def vector_form_t(m,I,t):
    n = pow(2,m)
    v = list()
    for i in range(n):
        v.append(f(to_binary_v(i,m)^t,I))
    return v

Hi = get_Hi(4,[0,1])
print("H_J = ")
print(Hi)

Ic = get_complimentary([0,1],list(range(4)))
print("J_c = ")
print(Ic)

vI = list()
for i in Hi:
    vI.append(vector_form_t(4,Ic,i))

print("v[2,3],H_j = ")
print(LinearCode(np.array(vI)))

H_J = 
[array([0, 0, 0, 0]), array([0, 0, 1, 0]), array([0, 0, 0, 1]), array([0, 0, 1, 1])]
J_c = 
[2, 3]
v[2,3],H_j = 
[1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]



In [238]:
def calc_mj(w,vI,r,m,j):
    wv = np.zeros((len(vI)))
    for i in range(len(vI)):
        wv[i] = mult(w,vI[i])
        # print("wv = ",wv[i])
        # print((vI[i]))
        # print(w)
    ones = np.sum(wv)
    zeroes = wv.shape[0]-ones
    
    if zeroes > pow(2,m-j-1):
        return 0
    elif ones > pow(2,m-j-1):
        return 1
    else:
        print("Parity error", ones, zeroes, len(vI))
        return -1

In [223]:
def check_mag_RM(w,r,m):
    u = list()
    w_i = w.copy()
    all_m_combs = get_all_combs(m)
    for i in range(r,-1,-1):
        
        J_list = list()
        m_j_list = list()
        
        for j in reversed(range(len(all_m_combs))):
            if len(all_m_combs[j])==i:
                J_list.append(all_m_combs[j])
        for j in range(len(J_list)):
            J_c = get_complimentary(J_list[j],list(range(m)))
            Hj = get_Hi(m,J_list[j])
            vj = list()
            for h in Hj:
                vj.append(vector_form_t(m,J_c,h))
            m_j_list.append(calc_mj(w_i,vj,r,m,i))
        u = list(reversed(m_j_list))+u
        vi = np.zeros((len(w_i)),dtype=int)
        for j in range(len(J_list)):
            vi ^= m_j_list[j]&np.array(vector_form(m,J_list[j]))
        w_i = w_i ^ vi
        if np.sum(w_i) <= pow(2,m-r,1)-1:
            return u
        
    return u

Исходное слово: 
[0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0]
Декодированное слово: 
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Обратно закодированное слово: 
[False  True False  True  True  True  True  True  True False  True False
 False False False False]
Совпадают ли?
[ True  True  True  True False  True  True  True  True  True  True  True
  True  True  True  True]


#### 5.3

In [225]:
w = [0,1,0,1,0,1,1,1,1,0,1,0,0,0,0,0]
u = check_mag_RM(w,2,4)

print("Исходное слово: ")
print(w) 

print("Декодированное слово: ")
print(u) 

print("Обратно закодированное слово: ")
w_decoded = mult(u,create_G_for_RM(2,4))
print(w_decoded)

print("Совпадают ли?")
print(w==w_decoded) 
# В исходном слове ошибка -- пример из лекции

Исходное слово: 
[0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0]
Декодированное слово: 
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Обратно закодированное слово: 
[False  True False  True  True  True  True  True  True False  True False
 False False False False]
Совпадают ли?
[ True  True  True  True False  True  True  True  True  True  True  True
  True  True  True  True]


In [226]:
u = [1,0,1,1,0,0,0,0,0,0,0]
G = create_G_for_RM(2,4)
w = mult(u,G)

u_dec = check_mag_RM(w,2,4)

print("Исходное слово: ")
print(u) 

print("Декодированное слово: ")
print(u_dec) 


print("Совпадают ли?")
print(u==u_dec) 
# Совпадают

Исходное слово: 
[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
Декодированное слово: 
[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
Совпадают ли?
True


In [229]:
errors = np.eye(16,dtype=bool)

u = [1,0,1,1,0,0,0,0,0,0,0]
G = create_G_for_RM(2,4)
w = mult(u,G)

w_err = w ^ errors[1]

u_dec = check_mag_RM(w_err,2,4)

print("Исходное слово: ")
print(u) 

print("Декодированное слово: ")
print(u_dec) 


print("Совпадают ли?")
print(u==u_dec) 
# Совпадают 1кратной ошибкой

Исходное слово: 
[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
Декодированное слово: 
[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
Совпадают ли?
True


In [245]:
errors = np.eye(16,dtype=bool)

u = [1,0,1,1,0,0,0,0,0,0,0]
G = create_G_for_RM(2,4)
w = mult(u,G)

w_err = w ^ errors[4] ^ errors[7]

u_dec = check_mag_RM(w_err,2,4)

print("Исходное слово: ")
print(u) 

print("Декодированное слово: ")
print(u_dec) 


print("Совпадают ли?")
print(u==u_dec) 
# С 2 кратными ошибками в голосовании 0 и 1 выходит ничья

Parity error 2.0 2.0 4
Parity error 2.0 2.0 4
Parity error 2.0 2.0 4
Parity error 2.0 2.0 4
Parity error 2.0 2.0 4
Исходное слово: 
[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
Декодированное слово: 
[1, 1, 1, 1, 0, -1, -1, -1, -1, -1, 0]
Совпадают ли?
False


In [244]:
errors = np.eye(16,dtype=bool)

u = [1,0,1,1,0,0,0,0,0,0,0]
G = create_G_for_RM(2,4)
w = mult(u,G)

w_err = w ^ errors[4] ^ errors[7]^ errors[9]

u_dec = check_mag_RM(w_err,2,4)

print("Исходное слово: ")
print(u) 

print("Декодированное слово: ")
print(u_dec) 


print("Совпадают ли?")
print(u==u_dec) 
# Не совпадают с 3 кратными ошибками

Исходное слово: 
[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
Декодированное слово: 
[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0]
Совпадают ли?
False
